In [1]:
%cd ..
import unified_dm
import logging
logging.getLogger("unified_dm").setLevel("ERROR")
from scripts.get_spreads import pull_spreads
from vbt_backtest.core import run_strategy_batch_single_thread, run_strategy
import pandas as pd
import itertools
from IPython.display import display

/home/stefano/development/crypto-arb


In [25]:
zscores = [1]
lookback_periods = [30]
results = []
for zscore, lookback_period in itertools.product(zscores, lookback_periods):

    def portfolio_stats(portfolios: pd.Series) -> pd.Series:
        return portfolios.apply(lambda s: pd.Series({"total_return": s.total_return(), "end_value": s.final_value()}))

    def show_details(index):
        from vbt_backtest.core import plot_spread_and_zscore

        feed = spreads.iloc[index].obj
        portfolio = portfolios.iloc[index]

        display(portfolio.plot_cum_returns())
        display(portfolio.stats())
        display(plot_spread_and_zscore(feed, portfolio))
        display(portfolio.trades.records_readable.sort_values("Entry Timestamp").head(10))
        display(portfolio.orders.records_readable.sort_values("Timestamp").head(20))
        display(feed.underlyings)

    print("zscore: ", zscore)
    print("lookback_period: ", lookback_period)
    spreads = pull_spreads(z_score_period=lookback_period, refresh_cache=False)
    print("running backtest...")
    portfolios = run_strategy_batch_single_thread(
        spreads, initial_cash=150000, trade_value=50000, z_score_thresh=zscore
    )
    print("computing stats...")
    stats = portfolio_stats(portfolios)
    res = pd.concat([spreads, stats], axis=1)
    res = res[["alias", "volatility", "missing_rows", "total_rows", "gaps"] + stats.columns.to_list()].sort_values(
        "total_return"
    )
    display(res.head(3), res.tail(3))
    print("mean total return: ", res.total_return.mean())
    print("median total return: ", res.total_return.median())
    print("mean end value: ", res.end_value.mean())
    print("median end value: ", res.end_value.median())
    results.append(
        {
            "zscore": zscore,
            "lookback_period": lookback_period,
            "mean_total_return": res.total_return.mean(),
            "median_total_return": res.total_return.median(),
            "mean_end_value": res.end_value.mean(),
            "median_end_value": res.end_value.median(),
        }
    )


WARNING                 unified_dm.util                                log:83   Using cached value in call to pull_spreads(z_score_period=30, good_spreads=True, minimum_rows=180): 0de77877777a8be71f82b5ff11dfba6d


zscore:  1
lookback_period:  30
running backtest...
computing stats...


,alias,volatility,missing_rows,total_rows,gaps,total_return,end_value
1003,gateio_PERPETUAL_kucoin_PERPETUAL_DOGE,3.808032,0,406,0,-0.277817,108327.485372
977,gateio_PERPETUAL_kucoin_PERPETUAL_BTC,1.880659,0,711,0,-0.134306,129854.142180
438,bybit_PERPETUAL_gateio_PERPETUAL_BTC,1.801099,0,716,0,-0.121303,131804.510166


,alias,volatility,missing_rows,total_rows,gaps,total_return,end_value
735,ftx_PERPETUAL_gateio_PERPETUAL_SXP,135.986275,0,578,0,3.682981,702447.097935
250,binance_PERPETUAL_gateio_PERPETUAL_SXP,135.857764,0,578,0,3.810797,721619.583721
822,okex_PERPETUAL_gateio_PERPETUAL_ANT,228.067817,0,566,0,4.729019,859352.806877


mean total return:  0.1440204952833492
median total return:  0.03557285728769387
mean end value:  171603.07429250237
median end value:  155335.92859315407


In [26]:
def show_details(index):
    from vbt_backtest.core import plot_spread_and_zscore

    feed = spreads.iloc[index].obj
    portfolio = portfolios.iloc[index]

    display(portfolio.plot_cum_returns())
    display(portfolio.stats())
    display(plot_spread_and_zscore(feed.set_index("open_time", drop=False), portfolio))
    display(portfolio.trades.records_readable.sort_values("Entry Timestamp").head(10))
    display(portfolio.orders.records_readable.sort_values("Timestamp").head(20))
    display(feed.underlyings)

In [32]:
spreads.iloc[1003].obj.iloc[80]

open_time    2021-04-19 00:00:00
open                    0.000413
high                    -0.00238
low                    -0.000149
close                  -0.000315
volume                 2136811.5
returns                -0.402034
zscore                 -0.258082
Name: 80, dtype: object

In [29]:
show_details(1003)

FigureWidget({
    'data': [{'line': {'color': '#7f7f7f'},
              'name': 'Benchmark',
              's…

Start                               2021-01-29 00:00:00
End                                 2022-03-10 00:00:00
Period                                406 days 00:00:00
Start Value                                    150000.0
End Value                                 108327.485372
Total Return [%]                             -27.781676
Benchmark Return [%]                         156.262301
Max Gross Exposure [%]                       506.468427
Total Fees Paid                             12113.83001
Max Drawdown [%]                              28.173334
Max Drawdown Duration                 326 days 00:00:00
Total Trades                                        141
Total Closed Trades                                 139
Total Open Trades                                     2
Open Trade PnL                               -45.716199
Win Rate [%]                                  48.201439
Best Trade [%]                               356.675667
Worst Trade [%]                             -477

ValueError: cannot reindex from a duplicate axis